# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119745e+02     1.534611e+00
 * time: 0.6858079433441162
     1     1.001989e+01     9.067000e-01
 * time: 1.9077880382537842
     2    -1.231277e+01     9.831490e-01
 * time: 2.4594340324401855
     3    -3.443302e+01     7.422740e-01
 * time: 3.296710968017578
     4    -4.795090e+01     5.380520e-01
 * time: 4.098114013671875
     5    -5.707327e+01     2.564140e-01
 * time: 4.9100000858306885
     6    -5.976870e+01     1.840169e-01
 * time: 5.471502065658569
     7    -6.080879e+01     5.363202e-02
 * time: 6.012686014175415
     8    -6.123132e+01     7.463058e-02
 * time: 6.555546045303345
     9    -6.150772e+01     6.634163e-02
 * time: 7.09138298034668
    10    -6.173787e+01     2.861154e-02
 * time: 7.646059036254883
    11    -6.192799e+01     2.190902e-02
 * time: 8.187505006790161
    12    -6.201083e+01     1.840368e-02
 * time: 8.728402137756348
    13    -6.208871e+01     1.830214e-02
 * time: 9.27440094947815
    1

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671062
    AtomicLocal         -18.8557671
    AtomicNonlocal      14.8522649
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485363 
    Xc                  -19.3336817

    total               -62.261666460927